In [51]:
import optuna

In [52]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error

from neuralforecast.auto import AutoLSTM
from neuralforecast.tsdataset import TimeSeriesDataset
from neuralforecast.core import NeuralForecast
from neuralforecast.models import LSTM
from datetime import datetime, timedelta
import plotly.graph_objects as go
import matplotlib.colors as mcolors
from neuralforecast.losses.pytorch import MQLoss

In [53]:
updated_df = pd.read_csv("https://raw.githubusercontent.com/cdcepi/FluSight-forecast-hub/refs/heads/main/target-data/target-hospital-admissions.csv")
updated_df = updated_df[updated_df["location_name"] == "US"]
updated_df = updated_df[["date", "value"]]
updated_df["date"] = pd.to_datetime(updated_df["date"])
updated_df.set_index("date", inplace = True)
updated_df.sort_values(by = "date", inplace = True)

updated_df["ds"] = updated_df.index
updated_df["unique_id"] = "series_1"
updated_df = updated_df.rename(columns = {"value": "y"})


In [54]:
input_df = updated_df.loc[:"2024-10-05"]
input_df

,y,ds,unique_id
date,,,
2022-02-05,1091.0,2022-02-05,series_1
2022-02-12,1158.0,2022-02-12,series_1
2022-02-19,1490.0,2022-02-19,series_1
2022-02-26,1603.0,2022-02-26,series_1
2022-03-05,1791.0,2022-03-05,series_1
...,...,...,...
2024-09-07,223.0,2024-09-07,series_1
2024-09-14,319.0,2024-09-14,series_1
2024-09-21,293.0,2024-09-21,series_1


In [55]:
train_size = int(len(input_df) * 0.7)
train_df = input_df.iloc[:train_size]
train_df

,y,ds,unique_id
date,,,
2022-02-05,1091.0,2022-02-05,series_1
2022-02-12,1158.0,2022-02-12,series_1
2022-02-19,1490.0,2022-02-19,series_1
2022-02-26,1603.0,2022-02-26,series_1
2022-03-05,1791.0,2022-03-05,series_1
...,...,...,...
2023-11-18,3491.0,2023-11-18,series_1
2023-11-25,4384.0,2023-11-25,series_1
2023-12-02,5948.0,2023-12-02,series_1


In [56]:
val_x = input_df.iloc[train_size:-4]
val_x

,y,ds,unique_id
date,,,
2023-12-23,15712.0,2023-12-23,series_1
2023-12-30,21674.0,2023-12-30,series_1
2024-01-06,19510.0,2024-01-06,series_1
2024-01-13,15890.0,2024-01-13,series_1
2024-01-20,13291.0,2024-01-20,series_1
2024-01-27,12986.0,2024-01-27,series_1
2024-02-03,11958.0,2024-02-03,series_1
2024-02-10,11626.0,2024-02-10,series_1
2024-02-17,11119.0,2024-02-17,series_1


In [57]:
val_y = input_df.iloc[train_size + 5:]
val_y

,y,ds,unique_id
date,,,
2024-01-27,12986.0,2024-01-27,series_1
2024-02-03,11958.0,2024-02-03,series_1
2024-02-10,11626.0,2024-02-10,series_1
2024-02-17,11119.0,2024-02-17,series_1
2024-02-24,10861.0,2024-02-24,series_1
2024-03-02,10806.0,2024-03-02,series_1
2024-03-09,9413.0,2024-03-09,series_1
2024-03-16,7321.0,2024-03-16,series_1
2024-03-23,6048.0,2024-03-23,series_1


In [58]:
windowed_train_df = create_windowed_df(train_df, input_size=5)

In [59]:
windowed_val_x = create_windowed_df(val_x, input_size=5)

In [60]:
windowed_val_y = create_windowed_df(val_y, input_size=4)

In [61]:
windowed_val_x

,y,ds,unique_id
0,15712.0,2023-12-23,window_000
1,21674.0,2023-12-30,window_000
2,19510.0,2024-01-06,window_000
3,15890.0,2024-01-13,window_000
4,13291.0,2024-01-20,window_000
...,...,...,...
165,275.0,2024-08-10,window_033
166,273.0,2024-08-17,window_033
167,274.0,2024-08-24,window_033
168,259.0,2024-08-31,window_033


In [62]:
windowed_val_y

,y,ds,unique_id
0,12986.0,2024-01-27,window_000
1,11958.0,2024-02-03,window_000
2,11626.0,2024-02-10,window_000
3,11119.0,2024-02-17,window_000
4,11958.0,2024-02-03,window_001
...,...,...,...
131,223.0,2024-09-28,window_032
132,319.0,2024-09-14,window_033
133,293.0,2024-09-21,window_033
134,223.0,2024-09-28,window_033


In [63]:
windowed_train_df

,y,ds,unique_id
0,1091.0,2022-02-05,window_000
1,1158.0,2022-02-12,window_000
2,1490.0,2022-02-19,window_000
3,1603.0,2022-02-26,window_000
4,1791.0,2022-03-05,window_000
...,...,...,...
465,3491.0,2023-11-18,window_093
466,4384.0,2023-11-25,window_093
467,5948.0,2023-12-02,window_093
468,7489.0,2023-12-09,window_093


In [64]:
def create_windowed_df(df, input_size, stride = 1):
    
    windows = []
    for i in range(0, len(df) - input_size + 1, stride):
        window = df.iloc[i:i+input_size].copy()
        window["unique_id"] = f"window_{i:03d}"
        windows.append(window)
    
    return pd.concat(windows, ignore_index=True)

In [65]:
def objective(trial):
    params = {
    #"encoder_hidden_size": trial.suggest_categorical("encoder_hidden_size", [50, 100, 150, 200, 250, 300, 350, 400, 450, 500]),
    #"encoder_n_layers": trial.suggest_categorical("encoder_n_layers", [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]),
    #"context_size": trial.suggest_categorical("context_size", [5, 10, 20, 30, 40, 50, 60, 70, 80]),
    #"decoder_hidden_size": trial.suggest_categorical("decoder_hidden_size", [16, 32, 64, 128, 256, 512]),
    "max_steps": 1000,
    #"batch_size": trial.suggest_categorical("batch_size", [1, 32]),
    "input_size": trial.suggest_categorical("input_size", [12, 16, 32, 64, 128, 256])
    }
    
    nf = NeuralForecast(models=[LSTM(h = 4, **params)], freq = "W-SAT")
    nf.fit(windowed_train_df)
    y_pred = nf.predict(df = windowed_val_x)
    
    return mean_absolute_error(windowed_val_y["y"], y_pred["LSTM"])

In [50]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=10)

[I 2025-06-18 13:52:49,748] A new study created in memory with name: no-name-71797be9-86b4-4a1f-9a2f-920d74125c32
[rank: 0] Seed set to 1
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name            | Type          | Params | Mode 
----------------------------------------------------------
0 | loss            | MAE           | 0      | train
1 | padder          | ConstantPad1d | 0      | train
2 | scaler          | TemporalNorm  | 0      | train
3 | hist_encoder    | LSTM          | 484 K  | train
4 | context_adapter | Linear        | 8.0 K  | train
5 | mlp_decoder     | MLP           | 2.4 K  | train
----------------------------------------------------------
494 K     Trainable params
0         Non-trainable params
494 K     Total params
1.978     Total estimated model params size (MB)
11        Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_steps=1000` reached.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

[I 2025-06-18 13:54:07,139] Trial 9 finished with value: 1049.9456671546486 and parameters: {'input_size': 32}. Best is trial 0 with value: 1049.9456671546486.


In [ ]:
study.best_params

In [ ]:
study.trials_dataframe()

In [ ]:
#Let's see how the predictions stack up against val_y
nf = NeuralForecast(models = [LSTM(h = 4, **study.best_params)], freq = "W-SAT")
nf.fit(windowed_train_df)
y_pred = nf.predict(df = windowed_val_x)


In [ ]:
y_pred

In [ ]:
windowed_val_y

In [ ]:
mean_absolute_error(windowed_val_y["y"], y_pred["LSTM"])

In [ ]:
#Now let's fit to the real input dataframe and predict
nf = NeuralForecast(models = [LSTM(h = 4, **study.best_params)], freq = "W-SAT")
nf.fit(input_df)

In [ ]:
preds = nf.predict(df = input_df)

In [ ]:
preds

In [ ]:
updated_df.loc["2024-10-12":"2024-11-02"]

In [ ]:
mean_absolute_error(updated_df.loc["2024-10-12":"2024-11-02"]["y"], preds["LSTM"])

In [80]:
nf = NeuralForecast(models = [LSTM(h = 4, max_steps=1000, batch_size = 32)], freq = "W-SAT")
nf.fit(windowed_train_df.iloc[:100])

[rank: 0] Seed set to 1
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name            | Type          | Params | Mode 
----------------------------------------------------------
0 | loss            | MAE           | 0      | train
1 | padder          | ConstantPad1d | 0      | train
2 | scaler          | TemporalNorm  | 0      | train
3 | hist_encoder    | LSTM          | 484 K  | train
4 | context_adapter | Linear        | 8.0 K  | train
5 | mlp_decoder     | MLP           | 2.4 K  | train
----------------------------------------------------------
494 K     Trainable params
0         Non-trainable params
494 K     Total params
1.978     Total estimated model params size (MB)
11        Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_steps=1000` reached.


In [67]:
windowed_train_df

,y,ds,unique_id
0,1091.0,2022-02-05,window_000
1,1158.0,2022-02-12,window_000
2,1490.0,2022-02-19,window_000
3,1603.0,2022-02-26,window_000
4,1791.0,2022-03-05,window_000
...,...,...,...
465,3491.0,2023-11-18,window_093
466,4384.0,2023-11-25,window_093
467,5948.0,2023-12-02,window_093
468,7489.0,2023-12-09,window_093


In [81]:
windowed_train_df

,y,ds,unique_id
0,1091.0,2022-02-05,window_000
1,1158.0,2022-02-12,window_000
2,1490.0,2022-02-19,window_000
3,1603.0,2022-02-26,window_000
4,1791.0,2022-03-05,window_000
...,...,...,...
465,3491.0,2023-11-18,window_093
466,4384.0,2023-11-25,window_093
467,5948.0,2023-12-02,window_093
468,7489.0,2023-12-09,window_093


In [76]:
windowed_train_df.iloc[:200]

,y,ds,unique_id
0,1091.0,2022-02-05,window_000
1,1158.0,2022-02-12,window_000
2,1490.0,2022-02-19,window_000
3,1603.0,2022-02-26,window_000
4,1791.0,2022-03-05,window_000
...,...,...,...
195,6400.0,2022-11-05,window_039
196,8579.0,2022-11-12,window_039
197,10940.0,2022-11-19,window_039
198,18149.0,2022-11-26,window_039
